In [2]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from PIL import Image
import os
from scipy import misc
from  collections import Counter


Using TensorFlow backend.


In [0]:
PATH_TO_IMAGES="/content/image_10_final/"
# PATH_TO_LABELS="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/files/_all_in_one_10.csv"

### Работа с данными

In [0]:
#Загрузка данных
X=np.load(PATH_TO_IMAGES+"X_data.npy")
y=np.load(PATH_TO_IMAGES+"y_data.npy")

In [0]:
#Кодирую имена числами
la=LabelEncoder()
y_copy=y.copy()
y=la.fit_transform(y)

In [0]:
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=17,stratify=y)

In [0]:
num_train, height, width,depth = X_train.shape 
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0] 

#new
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range


Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

### Создание архитектуры

In [0]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = (3,3) # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [0]:
inp = Input(shape=(height, width,depth))
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Conv2D(conv_depth_1, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Conv2D(conv_depth_1, kernel_size,  padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(drop_1)
conv_4 = Conv2D(conv_depth_2, kernel_size,  padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

### Обучение модели

In [10]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation


Train on 10076 samples, validate on 1120 samples
Epoch 1/200
10076/10076 [==============================] - 48s 5ms/step - loss: 7.7917 - acc: 9.9246e-05 - val_loss: 7.7935 - val_acc: 0.0000e+00
Epoch 2/200
 5728/10076 [================>.............] - ETA: 18s - loss: 7.7775 - acc: 5.2374e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7790 - acc: 2.9774e-04 - val_loss: 7.7990 - val_acc: 0.0000e+00
Epoch 3/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7761 - acc: 9.9246e-05 - val_loss: 7.8041 - val_acc: 0.0000e+00
Epoch 4/200
 1184/10076 [==>...........................] - ETA: 38s - loss: 7.7713 - acc: 8.4459e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7735 - acc: 1.9849e-04 - val_loss: 7.8090 - val_acc: 0.0000e+00
Epoch 5/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7710 - acc: 4.9623e-04 - val_loss: 7.8139 - val_acc: 0.0000e+00
Epoch 6/200
  128/10076 [....................

 9952/10076 [============================>.] - ETA: 0s - loss: 7.7429 - acc: 3.0145e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7429 - acc: 2.9774e-04 - val_loss: 7.9239 - val_acc: 0.0000e+00
Epoch 43/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7427 - acc: 1.9849e-04 - val_loss: 7.9256 - val_acc: 0.0000e+00
Epoch 44/200
 2112/10076 [=====>........................] - ETA: 34s - loss: 7.7439 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7426 - acc: 0.0000e+00 - val_loss: 7.9273 - val_acc: 0.0000e+00
Epoch 45/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7425 - acc: 1.9849e-04 - val_loss: 7.9288 - val_acc: 0.0000e+00
Epoch 46/200
  320/10076 [..............................] - ETA: 42s - loss: 7.7163 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7423 - acc: 1.9849e-04 - val_loss: 7.9300 - val_acc: 0.0000e+00
Epoch 47/200
 9

 2144/10076 [=====>........................] - ETA: 33s - loss: 7.7405 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7409 - acc: 2.9774e-04 - val_loss: 7.9627 - val_acc: 0.0000e+00
Epoch 85/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7409 - acc: 3.9698e-04 - val_loss: 7.9632 - val_acc: 0.0000e+00
Epoch 86/200
  352/10076 [>.............................] - ETA: 42s - loss: 7.7529 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7409 - acc: 9.9246e-05 - val_loss: 7.9637 - val_acc: 0.0000e+00
Epoch 87/200
 9952/10076 [============================>.] - ETA: 0s - loss: 7.7411 - acc: 1.0048e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7409 - acc: 9.9246e-05 - val_loss: 7.9642 - val_acc: 0.0000e+00
Epoch 88/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7409 - acc: 9.9246e-05 - val_loss: 7.9645 - val_acc: 0.0000e+00
Epoch 89/200
 2

  288/10076 [..............................] - ETA: 41s - loss: 7.7288 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 1.9849e-04 - val_loss: 7.9744 - val_acc: 0.0000e+00
Epoch 127/200
 9888/10076 [============================>.] - ETA: 0s - loss: 7.7412 - acc: 1.0113e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 9.9246e-05 - val_loss: 7.9744 - val_acc: 0.0000e+00
Epoch 128/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 1.9849e-04 - val_loss: 7.9746 - val_acc: 0.0000e+00
Epoch 129/200
 2080/10076 [=====>........................] - ETA: 34s - loss: 7.7307 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 0.0000e+00 - val_loss: 7.9748 - val_acc: 0.0000e+00
Epoch 130/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 1.9849e-04 - val_loss: 7.9749 - val_acc: 0.0000e+00
Epoch 131/2

 9920/10076 [============================>.] - ETA: 0s - loss: 7.7408 - acc: 4.0323e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 3.9698e-04 - val_loss: 7.9780 - val_acc: 0.0000e+00
Epoch 168/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 1.9849e-04 - val_loss: 7.9781 - val_acc: 0.0000e+00
Epoch 169/200
 2080/10076 [=====>........................] - ETA: 34s - loss: 7.7334 - acc: 4.8077e-0410076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 2.9774e-04 - val_loss: 7.9782 - val_acc: 0.0000e+00
Epoch 170/200
10076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 2.9774e-04 - val_loss: 7.9782 - val_acc: 0.0000e+00
Epoch 171/200
  288/10076 [..............................] - ETA: 41s - loss: 7.7223 - acc: 0.0000e+0010076/10076 [==============================] - 45s 4ms/step - loss: 7.7408 - acc: 2.9774e-04 - val_loss: 7.9782 - val_acc: 0.0000e+00
Epoch 172/2

### Проверка качества модели

In [11]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

4799/4799 [==============================] - 6s 1ms/step


[7.782045051986462, 0.000625130235465722]